In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from zipfile import ZipFile
import cv2
import pandas as pd

2023-12-14 20:41:24.279117: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-14 20:41:24.279182: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-14 20:41:24.325039: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-14 20:41:24.421165: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-14 20:41:25.398635: W tensorflow/compiler/tf2

In [2]:
# Define constants
width, height = 200, 200  # Adjust these dimensions based on your requirements
channels = 3  # Assuming RGB images
num_classes = 105  # Adjust based on the number of age classes in your dataset
num_epochs = 20
batch_size = 502


In [3]:
combined_faces_zip_path = "./assets/combined_faces.zip"
with ZipFile(combined_faces_zip_path, 'r') as myzip:
    myzip.extractall()
    print('Done unzipping combined_faces.zip')

Done unzipping combined_faces.zip


In [4]:

# Load the CSV file into a DataFrame
df = pd.read_csv('./assets/images_filenames_labels_test.csv')

# Display the first few rows of the DataFrame to understand the structure
print(df.head())

                              filename  age
0   ./content/combined_faces/8_163.jpg    8
1   ./content/combined_faces/38_66.jpg   38
2  ./content/combined_faces/40_177.jpg   40
3  ./content/combined_faces/36_267.jpg   36
4   ./content/combined_faces/8_349.jpg    8


In [5]:
# Extract image file paths and corresponding labels
file_paths = df['filename'].tolist()  # replace 'image_path_column' with the actual column name
labels = df['age'].tolist()  # replace 'age_column' with the actual column name


In [6]:
from sklearn.preprocessing import OneHotEncoder

# Reshape the labels to a 2D array
labels_reshaped = np.array(labels).reshape(-1, 1)

# Initialize the OneHotEncoder
encoder = OneHotEncoder()

# Fit and transform the labels to one-hot encoded vectors
one_hot_labels = encoder.fit_transform(labels_reshaped).toarray()

In [7]:
# Function to preprocess an image
def preprocess_image(image_path, target_size=(height, width)):
    img = cv2.imread(image_path)
    img = cv2.resize(img, target_size)
    img = img.astype('float32') / 255.0
    return img

preprocessed_images = [preprocess_image(path) for path in file_paths]
combined_data = list(zip(preprocessed_images, one_hot_labels))


In [8]:
# for img, label in combined_data[:5]:  # Display information for the first 5 samples
#     print(f"Image shape: {img.shape}, Label: {label}")

In [9]:
from sklearn.model_selection import train_test_split

# Assume combined_data is a list of tuples containing images and labels
# (preprocessed_images, one_hot_labels)
# Replace this with your actual combined_data variable

# Extract images and labels
images, labels = zip(*combined_data)

# Split the data into training, validation, and test sets
train_images, test_images, train_labels, test_labels = train_test_split(
    images, labels, test_size=0.2, random_state=42
)

# Further split the training data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42
)

# Print the sizes of the resulting datasets
print(f"Number of training samples: {len(train_images)}")
print(f"Number of validation samples: {len(val_images)}")
print(f"Number of test samples: {len(test_images)}")

Number of training samples: 6428
Number of validation samples: 1608
Number of test samples: 2010


In [10]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(height, width, channels)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

2023-12-14 20:41:47.168855: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-14 20:41:47.353757: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-14 20:41:47.353841: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-14 20:41:47.355924: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-14 20:41:47.355979: I external/local_xla/xla/stream_executor

In [14]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [15]:
print(f"Number of training samples: {len(train_images)}")
print(f"Number of training labels: {len(train_labels)}")
print(f"Number of val samples: {len(val_images)}")
print(f"Number of val labels: {len(val_labels)}")


print(train_images[0].shape)
print(val_images[0].shape)
print(test_images[0].shape)

Number of training samples: 6428
Number of training labels: 6428
Number of val samples: 1608
Number of val labels: 1608
(200, 200, 3)
(200, 200, 3)
(200, 200, 3)


In [16]:
model.fit(train_images, train_labels, epochs=num_epochs, batch_size=batch_size, validation_data=(val_images, val_labels))



In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test Accuracy: {test_acc}')


In [ ]:
img = cv2.imread('./image.jpg')
img = cv2.resize(img, (width, height))
img = img.astype('float32') / 255.0
img = np.expand_dims(img, axis=0)

In [ ]:
prediction = model.predict(img)
predicted_age = np.argmax(prediction)
print(f'Predicted Age: {predicted_age}')


In [ ]:
prediction = model.predict(img)
predicted_age = np.argmax(prediction)
print(f'Predicted Age: {predicted_age}')